In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!cp /content/drive/My\ Drive/Colab\ Notebooks/Student\ Queries/dataset-20200623T112641Z-001.zip /content/dataset.zip

In [ ]:
!unzip dataset.zip 

In [ ]:
from keras.applications import MobileNet

# MobileNet was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

# Re-loads the MobileNet model without the top or FC layers
MobileNet = MobileNet(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in MobileNet.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(MobileNet.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

Using TensorFlow backend.


0 InputLayer False
1 ZeroPadding2D False
2 Conv2D False
3 BatchNormalization False
4 ReLU False
5 DepthwiseConv2D False
6 BatchNormalization False
7 ReLU False
8 Conv2D False
9 BatchNormalization False
10 ReLU False
11 ZeroPadding2D False
12 DepthwiseConv2D False
13 BatchNormalization False
14 ReLU False
15 Conv2D False
16 BatchNormalization False
17 ReLU False
18 DepthwiseConv2D False
19 BatchNormalization False
20 ReLU False
21 Conv2D False
22 BatchNormalization False
23 ReLU False
24 ZeroPadding2D False
25 DepthwiseConv2D False
26 BatchNormalization False
27 ReLU False
28 Conv2D False
29 BatchNormalization False
30 ReLU False
31 DepthwiseConv2D False
32 BatchNormalization False
33 ReLU False
34 Conv2D False
35 BatchNormalization False
36 ReLU False
37 ZeroPadding2D False
38 DepthwiseConv2D False
39 BatchNormalization False
40 ReLU False
41 Conv2D False
42 BatchNormalization False
43 ReLU False
44 DepthwiseConv2D False
45 BatchNormalization False
46 ReLU False
47 Conv2D False
48 Batc

In [ ]:
def lw(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = Flatten()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(256,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

# Set our class number to 3 (Young, Middle, Old)
num_classes =2

FC_Head = lw(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/content/dataset/train/'
validation_data_dir = '/content/dataset/test/'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True)
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

train_generator.class_indices

Found 4778 images belonging to 2 classes.
Found 1223 images belonging to 2 classes.


{'cats': 0, 'yash': 1}

In [ ]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     


# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.00001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 250
nb_validation_samples = 1222

# We only train 5 EPOCHS 
epochs = 5
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples, 
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/5
250/250 [==============================] - 137s 546ms/step - loss: 0.0123 - accuracy: 0.9964 - val_loss: 6.7055e-08 - val_accuracy: 1.0000
Epoch 2/5
250/250 [==============================] - 131s 524ms/step - loss: 0.0022 - accuracy: 0.9996 - val_loss: 3.7253e-09 - val_accuracy: 1.0000
Epoch 3/5
250/250 [==============================] - 129s 518ms/step - loss: 3.8031e-04 - accuracy: 0.9999 - val_loss: 3.7253e-09 - val_accuracy: 1.0000
Epoch 4/5
250/250 [==============================] - 131s 525ms/step - loss: 2.2197e-04 - accuracy: 0.9999 - val_loss: 1.8626e-08 - val_accuracy: 1.0000
Epoch 5/5
250/250 [==============================] - 132s 527ms/step - loss: 3.0454e-04 - accuracy: 0.9999 - val_loss: 1.1176e-08 - val_accuracy: 1.0000


In [ ]:
model.save('yash_opt-rms_2class_category.h5')



In [ ]:
from keras.models import load_model

classifier = load_model('yash_opt-rms_2class_category.h5')

from keras.preprocessing import image
import numpy as np

In [ ]:
from keras.preprocessing import image
import numpy as np

In [ ]:
testc= image.load_img('/content/dataset/test/cats/cat.4008.jpg',target_size=(224,224))
testy = image.load_img('/content/dataset/test/yash/image101.jpg', target_size=(224, 224))

In [ ]:


testy=image.img_to_array(testy)
testy= np.expand_dims(testy, axis=0)


np.argmax(classifier.predict(testc, 1, verbose = 0),axis=0)

In [ ]:
testc.shape
testy.shape

(1, 224, 224, 3)

In [ ]:
c=classifier.predict(testc)
y=classifier.predict(testy)

In [ ]:
human = Face_detection_dict[str(pred)]

NameError: ignored

In [ ]:


classifier.predict(test, 1, verbose = 0)

In [ ]:
x=

In [ ]:
x.shape

In [ ]:
print(c,'\n\n', y)

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
import  cv2

In [1]:
cv2.imread('/content/dataset/test/cats/cat.4008.jpg')[:2,:2]

NameError: ignored

In [ ]:
timg[:2,:2]

In [ ]:
!free -mh

              total        used        free      shared  buff/cache   available
Mem:            12G        583M         10G        944K        2.1G         11G
Swap:            0B          0B          0B


/bin/bash: virt-host-validate: command not found
